<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/commons/f/f5/Best_Buy_Logo.svg" alt="bb logo" width="100"/>

<h1 style="color:#195071">BESTBUY Televisions</h1>

## Cleaning the Data 🧽

In [79]:
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
#!type python_proj\\bb_data.csv

#### load data as pd.DataFrame & set product 'sku' as index id

In [81]:
data = pd.read_csv('python_proj\\bb_data.csv')
data = data.set_index('sku')

In [82]:
data.shape

(465, 34)

In [83]:
data.head()

,brightness,color,curved,display_type,energy_eff,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,height_nostand,long_desc,manufacturer,model_num,model_year,online_avail,refresh_rate,regular_px,resolution,review_average,review_count,sale_px,screen_size,smart_capable,store_avail,sub_class,top_class,tv_name,warranty,weight_nostand,width
sku,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6466022,400 candela per square meter,Black,No,LED,149 kilowatt hours,Made For Outdoor With a weatherproof housing a...,Incredible Picture The 4K UHD screen delivers ...,Auto-Brightness Control The ambient light sens...,NaN,NaN,NaN,NaN,NaN,23.82 inches,Made for Outdoor Entertainment: Gather your fr...,Furrion,FDUF43CBS,2021.0,True,60Hz,1199.99,4K (2160p),2.5,2.0,1199.99,43 inches,No,True,"SO 30-44""TVs",PREMIUM FPTV,"Furrion - Aurora 43"" Full Shade 4K LED Outdoor TV",1 year,37.92 pounds,39.88 inches
6414244,NaN,Black,No,LED,182 kilowatt hours,Talk to Your Customers The slideshow feature a...,Not a Content Expert? No Problem. With the int...,Create Eye-Catching Content With the Samsung B...,Plug and Play the Content You Want Show what y...,A TV You Can Count On Featuring an impressive ...,NaN,NaN,NaN,37.8 inches,"With the Samsung Pro TV, content becomes as dy...",Samsung,BE70T-H,2020.0,True,60Hz,969.00,4K (2160p),1.7,3.0,969.00,70 inches,No,True,SO COMMERCIAL TV,SMALL FPTV 0-31,"Samsung - 70"" CLASS BE70T-H LED 4K Commercial ...",3 year limited,54.5 pounds,61.9 inches
6461913,NaN,Gray,No,LED,318 kilowatt hours,QNED Mini-LED Display LG Quantum Dot NanoCell ...,Quantum Dot NanoCell Color Technology Two colo...,Mini-LED Backlight Tiny LEDs offer a more deta...,TruMotion 240 (Native 120Hz) TruMotion 240 tec...,Dolby Vision IQ and Dolby Atmos With Dolby Vis...,Full Array Dimming Pro Full Array Dimming Pro'...,a7 Gen 4 AI Processor 4K Catch every detail wi...,ThinQ AI Control your LG smart products and ot...,43.2 inches,Synergy in light and color and smooth action o...,LG,86QNED90UPA,2021.0,True,120Hz,3999.99,4K (2160p),1.0,3.0,3499.99,86 inches,Yes,True,"MHT 61""-100"" TV's",PREMIUM FPTV,"LG - 86"" Class 90 Series QNED Mini-LED 4K UHD ...",1 Year,110.9 pounds,75.5 inches
6466017,750 candela per square meter,Black,No,LED,188 kilowatt hours,Made For Outdoor With a weatherproof housing a...,Incredible Picture The 4K UHD screen delivers ...,Auto-Brightness Control The ambient light sens...,NaN,NaN,NaN,NaN,NaN,29.75 inches,Made for Outdoor Entertainment: Gather your fr...,Furrion,FDUP55CBS,2021.0,True,60Hz,2599.99,4K (2160p),1.0,1.0,2599.99,55 inches,No,False,SO MHT 45-100 LTL,PREMIUM FPTV,"Furrion - Aurora 55"" Partial Sun 4K LED Outdoo...",1 year,60.6 pounds,50.2 inches
5900939,500 candela per square meter,Black,No,LED,173 kilowatt hours,Waterproof universal remote Can control up to ...,4K Ultra HD (2160p resolution) Enjoy breathtak...,Watch high dynamic range (HDR) content on your...,LED TV performs well in most lighting conditio...,Advanced TV sound Two 5W speakers.,Enjoy the picture from multiple angles 178&#17...,NaN,NaN,26.7 inches,Keep barbeque guests entertained with this 49-...,Peerless-AV,UV492,NaN,False,NaN,2999.99,4K (2160p),NaN,NaN,2999.99,49 inches,No,True,SO MHT 45-100 LTL,PREMIUM FPTV,"Peerless-AV - 49"" Class LED Outdoor Full Sun 4...",2 years,52.8 pounds,45.1 inches


#### Separate the text data not used in the current quantitative analysis

In [84]:
# Separate language data into separate DataFrame (product description, features, etc)
col_words = ['long_desc']+['feat'+str(i+1) for i in range(8)]
data_words = data.loc[:,data.columns.isin(col_words+['tv_name'])]

# Remove the copied word columns from main data, except tv_name
data = data.loc[:,~data.columns.isin(col_words)]

- Save to csv "python_proj\\bb_words.csv"

In [85]:
data_words.to_csv('python_proj\\bb_words.csv')

In [86]:
# for columns "curved", "smart capable", convert Yes / No to boolean values
data[['curved','smart_capable']] = data[['curved','smart_capable']].applymap(lambda x: True if x.lower() == 'yes' else False)

#### Most of the quantitative data still needs to be extracted and set as int or float type

In [87]:
# set column dtype as Int64 to conver to int datatype
target_cols = ['energy_eff','refresh_rate','regular_px','sale_px','screen_size',
               'height_nostand','weight_nostand','width']

data[target_cols] = data[target_cols].applymap(lambda x: re.sub(r'[^.0-9]','',str(x)))

In [88]:
# input np.nan for any empty strings
data[target_cols] = data[target_cols].applymap(lambda x: np.nan if x == '' else x)

In [89]:
# some columns should be Int64 dtype

intcols = ['model_year','refresh_rate','review_count','screen_size']
data[intcols] = data[intcols].convert_dtypes()

In [90]:
# while others should be float
floatcols = ['energy_eff','regular_px','sale_px','screen_size',
             'height_nostand','weight_nostand','width']
data[floatcols] = data[floatcols].applymap(lambda x: float(x))

#### Dealing with the Warranty column
some are stated in years, some in months, and some days<br>
--> convert all to **years**

some have different warranty periods for different parts<br>
--> take **shortest** period

In [91]:
# define function to filter years / months / days data, and convert all to YEARS
def warr_func(arr):
    '''
    arr = a 1D array or pandas series
    returns arr after converting all numerical values to years (integer)
    '''
    def get_val(v):
        # removes all non-numerical chars
        vs = re.sub(r'[^ 0-9]','',str(v)).split()
        return int(min(vs))
    
    for i in arr.index:
        if re.search('month',arr[i].lower()):    # months to years
            arr[i] = get_val(arr[i])/12
        elif re.search('year',arr[i].lower()):  # years to years
            arr[i] = get_val(arr[i])
        elif int(arr[i]) > 100:                  # days to years
            arr[i] = int(arr[i])/364
    
    return arr

data['warranty'] = warr_func(data['warranty'].copy())

#### state original units in column names
- energy efficiency: KW hours per year 
- refresh rate: Hz
- all others were either in dollars or inches

In [92]:
data.rename(columns={'energy_eff': 'energy_KWh',
                     'refresh_rate': 'refresh_Hz',
                     'warranty': 'warranty_yrs'},
            inplace=True)

<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/commons/f/f5/Best_Buy_Logo.svg" alt="bb logo" width="100"/>

## Checking Data Validity 🔍

#### Drop columns with too many missing values or other unused columns

In [93]:
# Drop 'brightness' column, since 409 out of 465 are missing value

data['brightness'].isnull().sum()  #409

data = data.drop('brightness',axis=1)

#### Remove TVs that are outdated -OR- not currently sold (both online or in-store)

In [94]:
data[~data['online_avail'] & ~data['store_avail']].count()['tv_name']  # 53
data[data['model_year'] < 2019].count()['tv_name']  # 16
data[(data['model_year'] < 2019) | (~data['online_avail'] & ~data['store_avail'])].count()['tv_name']

68

In [95]:
# Remove these 68 rows
data = data.drop(data[(data['model_year'] < 2019) | (~data['online_avail'] & ~data['store_avail'])].index)

## Rebucketing Sizes 🗑️

Looking at TV categorization from bestbuy:<br>
there are overlapping classes of TVs, e.g. *PREMIUM FPTV - 30-50"* <> *LARGE FPTV - 45-50"*<br>
there are also **mistakes** where 40-60" TVs in the 'SMALL FPTV' class<br>
- lets rebucket based on screen_size and use their cutoffs

        SMALL  == < 31"
        MID    == 32"-45"
        LARGE  == 45"-60"
        XLARGE == > 60"

In [96]:
data.groupby(['top_class','sub_class']).agg(['count','min','max','median','mean'])['screen_size']

count   min   max  median       mean
top_class        sub_class                                               
LARGE FPTV 46-60 45"-50"  TV's          23  50.0  50.0    50.0  50.000000
                 51"-55" TV's           27  55.0  55.0    55.0  55.000000
                 56"-60" TV's            8  58.0  60.0    59.0  59.000000
MDC HARDWARE     MDC TV (VC)             1  88.0  88.0    88.0  88.000000
PREMIUM FPTV     MHT 30"- 50"           15  32.0  50.0    48.0  45.866667
                 MHT 51" - 60" TV's     25  55.0  55.0    55.0  55.000000
                 MHT 61"-100" TV's      99  65.0  86.0    75.0  74.121212
                 SO 30-44"TVs            2  43.0  43.0    43.0  43.000000
                 SO MHT 45-100 LTL      18  43.0  75.0    55.0  58.333333
SMALL & MID FPTV MID 32" LCD            17  32.0  32.0    32.0  32.000000
                 MID 40"-45" LCD        33  39.0  43.0    43.0  42.303030
                 SMALL TV 0-31"          9  19.0  28.0    24.0  23.888889
SMALL FPTV 0-31  SO COMMERCIAL TV        5  55.0  82.0    75.0  71.400000
                 SO SMALL TV 0-31"       2  43.0  50.0    46.5  46.500000
XLARGE FPTV 61+  61-74" TVS             56  65.0  70.0    65.0  66.517857
                 75"+ TVS               57  70.0  86.0    75.0  77.877193

In [97]:
size_class = pd.cut(data['screen_size'],bins=[0,32,45,60,150],labels=['SMALL','MID','LARGE','XLARGE'])
data.insert(15,'size_class',size_class)

- Check mean sizes of each new category

In [98]:
data.groupby('size_class').agg('mean')['screen_size']

size_class
SMALL     29.296296
MID       42.452381
LARGE     53.476190
XLARGE    73.147982
Name: screen_size, dtype: float64

## Rebucketing Brands (Manufacturer) 🗑️

Looking at the different manufacturers:<br>
The top 7 brands have about 2 dozen models of TVs, with Bestbuy's own Insignia bottoming-out the list<br>
The rest are smaller brands with less than 12
- lets combine the bottom brands into "others"
        ---Others---
        Toshiba    Furrion   Peerless-AV   SunBriteTV   Amazon   Westinghouse   Pioneer

In [99]:
data.groupby(['manufacturer']).agg('count')['tv_name'].sort_values().head(7)
# total 39 TVs in new 'manufacturer' category "Others"

manufacturer
Pioneer          1
Westinghouse     2
Amazon           4
Peerless-AV      5
SunBriteTV       5
Furrion         10
Toshiba         12
Name: tv_name, dtype: int64

In [100]:
data.groupby(['manufacturer']).agg('count')['tv_name'].sort_values().head(7).sum()
# total 39 TVs in new 'manufacturer' category "Others"

39

In [101]:
to_bin = ['Toshiba','Furrion','Peerless-AV','SunBriteTV','Amazon','Westinghouse','Pioneer']
for idx, each in data.manufacturer.iteritems():
    if each in to_bin:
        data.loc[idx,'manufacturer'] = 'Others'

In [102]:
data.groupby(['manufacturer']).agg('count')['tv_name'].sort_values(ascending=False)

manufacturer
Samsung      115
LG            83
Sony          49
Others        39
TCL           33
VIZIO         30
Hisense       25
Insignia™     23
Name: tv_name, dtype: int64

## Calculate the "Bezel"  📺

Bezel is the gap from the viewable screen to the actual sides of the TV<br>
For most, a *thin* bezel is preferable<br>

    Calculating the Bezel as a ratio:
    'screen_size' / sqrt(height_nostand^2 + width^2)
   
**The closer it is to 1, the better**

In [103]:
data['bezel'] = data.screen_size / np.sqrt(data.height_nostand**2 + data.width**2)

In [104]:
# a few resulted in >1... set those to 0.999985 (2nd highest after >1)
data.sort_values(by='bezel',ascending=False)['bezel']
for i in data[data.bezel>1].index:
    data.loc[i,'bezel'] = 0.999985

In [105]:
data.head()

,color,curved,display_type,energy_KWh,height_nostand,manufacturer,model_num,model_year,online_avail,refresh_Hz,regular_px,resolution,review_average,review_count,sale_px,size_class,screen_size,smart_capable,store_avail,sub_class,top_class,tv_name,warranty_yrs,weight_nostand,width,bezel
sku,,,,,,,,,,,,,,,,,,,,,,,,,,
6466022,Black,False,LED,149.0,23.82,Others,FDUF43CBS,2021,True,60,1199.99,4K (2160p),2.5,2,1199.99,MID,43.0,False,True,"SO 30-44""TVs",PREMIUM FPTV,"Furrion - Aurora 43"" Full Shade 4K LED Outdoor TV",1,37.92,39.88,0.925683
6414244,Black,False,LED,182.0,37.80,Samsung,BE70T-H,2020,True,60,969.00,4K (2160p),1.7,3,969.00,XLARGE,70.0,False,True,SO COMMERCIAL TV,SMALL FPTV 0-31,"Samsung - 70"" CLASS BE70T-H LED 4K Commercial ...",3,54.50,61.90,0.965132
6461913,Gray,False,LED,318.0,43.20,LG,86QNED90UPA,2021,True,120,3999.99,4K (2160p),1.0,3,3499.99,XLARGE,86.0,True,True,"MHT 61""-100"" TV's",PREMIUM FPTV,"LG - 86"" Class 90 Series QNED Mini-LED 4K UHD ...",1,110.90,75.50,0.988670
6466017,Black,False,LED,188.0,29.75,Others,FDUP55CBS,2021,True,60,2599.99,4K (2160p),1.0,1,2599.99,LARGE,55.0,False,False,SO MHT 45-100 LTL,PREMIUM FPTV,"Furrion - Aurora 55"" Partial Sun 4K LED Outdoo...",1,60.60,50.20,0.942535
5900939,Black,False,LED,173.0,26.70,Others,UV492,<NA>,False,<NA>,2999.99,4K (2160p),NaN,<NA>,2999.99,LARGE,49.0,False,True,SO MHT 45-100 LTL,PREMIUM FPTV,"Peerless-AV - 49"" Class LED Outdoor Full Sun 4...",2,52.80,45.10,0.934920


## Save to bb_clean.csv 💾

In [106]:
data.to_csv('python_proj\\bb_clean.csv')